In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
! gcloud config set project main-train-445501

Updated property [core/project].


In [ ]:
!gcloud ai models describe 4054423838656561152

Please specify a region:
 [1] africa-south1
 [2] asia-east1
 [3] asia-east2
 [4] asia-northeast1
 [5] asia-northeast2
 [6] asia-northeast3
 [7] asia-south1
 [8] asia-southeast1
 [9] asia-southeast2
 [10] australia-southeast1
 [11] australia-southeast2
 [12] europe-central2
 [13] europe-north1
 [14] europe-southwest1
 [15] europe-west1
 [16] europe-west2
 [17] europe-west3
 [18] europe-west4
 [19] europe-west6
 [20] europe-west8
 [21] europe-west9
 [22] europe-west12
 [23] me-central1
 [24] me-central2
 [25] me-west1
 [26] northamerica-northeast1
 [27] northamerica-northeast2
 [28] southamerica-east1
 [29] southamerica-west1
 [30] us-central1
 [31] us-east1
 [32] us-east4
 [33] us-east5
 [34] us-south1
 [35] us-west1
 [36] us-west2
 [37] us-west3
 [38] us-west4
 [39] cancel
Please enter your numeric choice:  30

To make this the default region, run `gcloud config set ai/region us-central1`.

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
containerSpec:
  healthRoute: /h

In [ ]:
%%writefile main.py

from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from pytorch_tabnet.tab_model import TabNetRegressor
from pydantic import BaseModel
from typing import List, Optional
import numpy as np
import joblib
import pandas as pd
import os

app = FastAPI(title="Student Grade Prediction API")

# Define API routes
AIP_HEALTH_ROUTE = os.environ.get('AIP_HEALTH_ROUTE', '/health')
AIP_PREDICT_ROUTE = os.environ.get('AIP_PREDICT_ROUTE', '/predict')
MODEL_INFO_ROUTE = '/model-info'
FEATURES_ROUTE = '/features'

# Load saved model and preprocessor
model = TabNetRegressor()
model.load_model("model8.zip")

class StudentData(BaseModel):
    namsinh: int
    dtb_toankhoa: float
    dtb_tichluy: float
    sotc_tichluy: int
    diemtbhk_1: float
    diemtbhk_2: float
    diemtbhk_3: float
    diemtbhk_4: float
    diemtbhk_5: float
    diemtbhk_6: float
    diemtbhk_7: float

class PredictionRequest(BaseModel):
    instances: List[StudentData]

class PredictionResponse(BaseModel):
    predicted_grade: float
    confidence_score: Optional[float]

class Predictions(BaseModel):
    predictions: List[PredictionResponse]

# Health check endpoint
@app.get(AIP_HEALTH_ROUTE, status_code=200)
async def health():
    return {'healthy': 'ok'}

# Model information endpoint
@app.get(MODEL_INFO_ROUTE)
async def model_info():
    return {
        'model_type': 'TabNet Regressor',
        'feature_count': len(feature_names),
        'target_variable': 'diemtbhk_8',
    }

# Available features endpoint
@app.get(FEATURES_ROUTE)
async def get_features():
    return {
        'features': feature_names.tolist(),
        'required_features': StudentData.schema()['required']
    }

# Batch prediction endpoint
@app.post(AIP_PREDICT_ROUTE, response_model=Predictions)
async def predict(request: Request):
      # Get JSON body
      body = await request.json()
      feature_names = ['namsinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1', 'diemtbhk_2', 'diemtbhk_3', 'diemtbhk_4', 'diemtbhk_5', 'diemtbhk_6', 'diemtbhk_7']

      # Convert instances to StudentData objects
      instances = [StudentData(**inst) for inst in body["instances"]]
      input_data = pd.DataFrame([inst.dict() for inst in instances])

      # Ensure columns are in correct order
      input_data = input_data[feature_names]

      # Make predictions
      predictions = model.predict(input_data.values)
      return {"predictions": predictions.tolist()}

# Error handler
@app.exception_handler(Exception)
async def general_exception_handler(request: Request, exc: Exception):
    return JSONResponse(
        status_code=500,
        content={
            "error": str(exc),
            "type": type(exc).__name__
        }
    )

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

Overwriting main.py


In [ ]:
%%writefile Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.8-slim
RUN pip install --no-cache-dir  numpy==1.23.1 pytorch-tabnet fastapi pydantic joblib pandas uvicorn
COPY main.py ./main.py
COPY model8.zip ./model8.zip

Overwriting Dockerfile


In [ ]:
%%writefile cloudbuild.yaml

steps:
# Bước 1: Tải mô hình từ Google Cloud Storage
- name: 'gcr.io/cloud-builders/gsutil'
  args: ['cp', 'gs://student_train/model/model8.zip', '.']
  id: 'download-model'

# Bước 2: Build Docker Image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/main-train-445501/predict-neww', '.']
  waitFor: ['download-model']

# Bước 3: Đẩy Docker Image lên Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/main-train-445501/predict-neww']

images:
- gcr.io/main-train-445501/predict-neww

Overwriting cloudbuild.yaml


In [ ]:
!gcloud builds submit --config cloudbuild.yaml

Creating temporary archive of 62 file(s) totalling 56.8 MiB before compression.
Uploading tarball of [.] to [gs://main-train-445501_cloudbuild/source/1734969686.277068-15f2bbe17ada4e0a9dfb9a310893bbd0.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/main-train-445501/locations/global/builds/95a1eff6-af0f-418a-8331-1791e72cd00d].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/95a1eff6-af0f-418a-8331-1791e72cd00d?project=990238876285 ].
Waiting for build to complete. Polling interval: 1 second(s).
 REMOTE BUILD OUTPUT
starting build "95a1eff6-af0f-418a-8331-1791e72cd00d"

FETCHSOURCE
Fetching storage object: gs://main-train-445501_cloudbuild/source/1734969686.277068-15f2bbe17ada4e0a9dfb9a310893bbd0.tgz#1734969696379913
Copying gs://main-train-445501_cloudbuild/source/1734969686.277068-15f2bbe17ada4e0a9dfb9a310893bbd0.tgz#1734969696379913...
/ [1 files][  7.5 MiB/  7.5 MiB]                                                
Operation completed over 1 o

In [ ]:
!gcloud ai models upload \
  --container-ports=80 \
  --container-predict-route="/predict" \
  --container-health-route="/health" \
  --region=us-central1 \
  --display-name=student-predictt \
  --container-image-uri=gcr.io/main-train-445501/predict-neww

Using endpoint [https://us-central1-aiplatform.googleapis.com/]


In [ ]:
!gcloud ai endpoints create \
  --project=main-train-445501 \
  --region=us-central1 \
  --display-name=student-predictt

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Created Vertex AI endpoint: projects/990238876285/locations/us-central1/endpoints/970993012140146688.


In [ ]:
!gcloud ai endpoints deploy-model 970993012140146688 \
  --project=main-train-445501 \
  --region=us-central1 \
  --model=3052372921566625792 \
  --traffic-split=0=100 \
  --machine-type="n1-standard-2" \
  --display-name=studentt

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Deployed a model to the endpoint 970993012140146688. Id of the deployed model: 4026874475311005696.


In [ ]:
%%writefile request.json
{
  "instances": [
    {
      "namsinh": 19,
      "dtb_toankhoa": 9.37,
      "dtb_tichluy": 9.37,
      "sotc_tichluy": 24,
      "diemtbhk_1": 9.37,
      "diemtbhk_2": 9.34,
      "diemtbhk_3": 9.4,
      "diemtbhk_4": 9.37,
      "diemtbhk_5": 9.37,
      "diemtbhk_6": 9.37,
      "diemtbhk_7": 9.37
    }
  ]
}

Overwriting request.json


In [ ]:
!gcloud ai endpoints predict 2613680976223535104 \
  --region=us-central1 \
  --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
ERROR: (gcloud.ai.endpoints.predict) HttpError accessing <https://us-central1-prediction-aiplatform.googleapis.com/v1/projects/main-train-445501/locations/us-central1/endpoints/2613680976223535104:predict?alt=json>: response: <{'date': 'Mon, 23 Dec 2024 15:56:12 GMT', 'content-type': 'application/json', 'x-vertex-ai-internal-prediction-backend': 'harpoon', 'vary': 'Origin, X-Origin, Referer', 'content-encoding': 'gzip', 'server': 'scaffolding on HTTPServer2', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'transfer-encoding': 'chunked', 'status': 500}>, content <{"error":"1 validation error for Predictions\nresponse\n  value is not a valid dict (type=type_error.dict)","type":"ValidationError"}>
This may be due to network connectivity issues. Please check your network settings, and the status of the service you are trying to reach.


In [ ]:
import json
import pandas as pd

In [ ]:
# !pip install pytorch_tabnet

In [ ]:
# from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
# with open('/content/request.json', 'r') as file:
#     body = json.load(file)  # Đọc file JSON thành dict

# # Danh sách các đặc trưng
# feature_names = ['namsinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy',
#                  'diemtbhk_1', 'diemtbhk_2', 'diemtbhk_3', 'diemtbhk_4',
#                  'diemtbhk_5', 'diemtbhk_6', 'diemtbhk_7']

# # Chuyển đổi dữ liệu thành đối tượng StudentData
# instances = [StudentData(**inst) for inst in body["instances"]]
# input_data = pd.DataFrame([inst.dict() for inst in instances])

# # Ensure columns are in correct order
# input_data = input_data[feature_names]
# model = TabNetRegressor()
# model.load_model("model8.zip")

# # Make predictions
# predictions = model.predict(input_data.values)
# predictions

<ipython-input-90-a09cde8478b3>:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  input_data = pd.DataFrame([inst.dict() for inst in instances])
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the fu

array([[9.67573]], dtype=float32)

In [ ]:
# return {"predictions": predictions.tolist()}

predictions [[9.675729751586914]]


In [ ]:
# input_data.values

array([[19.  ,  9.37,  9.37, 24.  ,  9.37,  9.34,  9.4 ,  9.37,  9.37,
         9.37,  9.37]])

In [ ]:
student_data = {
    "namsinh": 10,
    "dtb_toankhoa": 8.0,
    "dtb_tichluy": 8.2,
    "sotc_tichluy": 120,
    "diemtbhk_1": 8.3,
    "diemtbhk_2": 8.4,
    "diemtbhk_3": 8.5,
    "diemtbhk_4": 8.6,
    "diemtbhk_5": 8.7,
    "diemtbhk_6": 8.8,
    "diemtbhk_7": 8.9
}

predict_custom_trained_model_sample(
    project="990238876285",
    endpoint_id="970993012140146688",
    location="us-central1",
    instances=student_data
)

InternalServerError: 500 {"error":"1 validation error for Predictions\nresponse -> predictions -> 0\n  value is not a valid dict (type=type_error.dict)","type":"ValidationError"}

In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START aiplatform_predict_custom_trained_model_sample]
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", prediction)


# [END aiplatform_predict_custom_trained_model_sample]